**Setup (for Windows 10)**

1. Create a Python 3.7 'pytorch' environment with Anaconda Navigator
2. Activate the environment and run 'install pytorch torchvision - c pytorch': https://medium.com/@iamHarin17/how-to-setup-a-python-environment-for-deep-learning-with-anaconda-f65ab78a362
3. Run 'pip install' for 'matplotlib', 'tqdm' and 'scipy'
4. Install Jupyter Notebook from Anaconda Navigator and open this notebook.

**Helpful Links**

DOS Nvidia GPU usage: https://wiki.tuflow.com/index.php?title=DOS_GPU_Usage

cd C:\Program Files\NVIDIA Corporation\NVSMI

In [1]:
# Set global variables to run notebook

# data root
data_root = 'C:\\_data\\' # set to whever the data is being stored

# hyperparameters
# with m2200 - 15min with 8, 10min with 16, 9min with 24 and out of memory with 32
global_batch_size = 16 # originally set to 4, but we've agreed to use 16 as default

# parameters to control length of training
use_rekall_labels = True # False will use 'true' labels - files need to be in 'commercials\\data\\'
data_percentage = .2     # percentage of training data to us
num_seeds = 1            # number of different seeds to run, originally 5
global_num_epochs = 25   # originally 25

# BrokenPipeError on Windows 10
# https://discuss.pytorch.org/t/brokenpipeerror-errno-32-broken-pipe-when-i-run-cifar10-tutorial-py/6224/3
global_num_workers = 0 # originally set to 4, so perhaps should be 4 on Mac

In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import random
from PIL import Image
from tqdm import tqdm
from scipy import stats

In [3]:
device = torch.device('cuda:0')

In [4]:
# data_transforms = {
#     'train': transforms.Compose([
# #         transforms.Resize((310, 310)),
#         transforms.Resize((224, 224)),
# #         transforms.CenterCrop(224),
# #         transforms.RandomResizedCrop(224),
# #         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
# #         transforms.Resize((310, 310)),
#         transforms.Resize((224, 224)),
# #         transforms.Resize(256),
# #         transforms.CenterCrop(224),
# #         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
class ImageDataset(Dataset):
    
    def __init__(self, paths, labels, transform=None, stride = 1, max_size=None):
        self.paths = paths[::stride]
        if max_size is not None:
            self.paths = self.paths[:max_size]
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_name = self.paths[idx]
        img = Image.open(img_name).convert('RGB')        
        img_tensor = self.transform(img)
        return img_tensor, self.labels[idx]

In [6]:
if use_rekall_labels: # use Rekall label for training
    train_file = os.path.join(data_root, 'commercials\\data\\rekall_train.txt')
else:
    train_file = os.path.join(data_root, 'commercials\\data\\train.txt')

# continue to use true labels for validation and test
val_file   = os.path.join(data_root, 'commercials\\data\\val.txt')
test_file  = os.path.join(data_root, 'commercials\\data\\test.txt')

print('Training with ', train_file)

Training with  C:\_data\commercials\data\rekall_train.txt


In [7]:
def read_file(filename, load_percentage = 1):
    paths = []
    labels = []
    videos = []
    
    with open(filename, 'r') as f:
        
        # find set of unique videos on file
        for line in f.readlines():
            video, image, label = line.split(' ')
            videos.append(video)
        videos = set(videos) # set of all videos
        sorted(videos) # make sure they're in the same order each time, although they're alreayd sorted in files
        # reduce set of unique videos by load percentage
        num_videos = int(len(set(videos)) * load_percentage) # number of videos to use
        videos = list(videos)[:num_videos] # take first 'num_videos'
        f.seek(0) # go back to beginning of file
        
        # read in data only using videos in possibly reduced set from above
        for line in f.readlines():
            video, image, label = line.split(' ')
            if video in videos: # load it up only if it's in the list of videos to use
                paths.append(data_root + 'commercials\\images\\{}\\{:04d}.jpg'.format(video, int(image)))
                labels.append(int(label))
            
    return paths, labels

In [8]:
train_paths, Y_train = read_file(train_file, data_percentage)
val_paths  , Y_val   = read_file(val_file)
test_paths , Y_test  = read_file(test_file)

# https://stackoverflow.com/a/32577022/852795
# train_paths, Y_train = zip(*random.sample(list(zip(train_paths, Y_train)), int(len(train_paths) * data_percentage)))

In [9]:
# val_train is the same as val
image_datasets = {
    'train'    : ImageDataset(train_paths, Y_train, transform = data_transforms['train'], stride=1, max_size=None),
    'val_train': ImageDataset(val_paths  , Y_val  , transform = data_transforms['train'], stride=1, max_size=None),
    'val'      : ImageDataset(val_paths  , Y_val  , transform = data_transforms['val']  , stride=1, max_size=None),
    'test'     : ImageDataset(test_paths , Y_test , transform = data_transforms['val']  , stride=1, max_size=None)
}

In [10]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size = global_batch_size,
                                              shuffle = 'train' in x,
                                              num_workers = global_num_workers,
                                              pin_memory = True)
                                              for x in image_datasets}

In [11]:
dataset_sizes = {
    x: len(image_datasets[x]) for x in image_datasets
}
dataset_sizes

{'train': 12955, 'val_train': 9479, 'val': 9479, 'test': 7496}

In [12]:
def safe_divide(a, b):
    return a / b if b > 0 else 0

In [13]:
def train_model(model, criterion, optimizer, scheduler, train_dl, val_dl, test_dl=None,
                num_epochs=25, return_best=False, verbose=True, log_file=None):
    print(train_dl, val_dl, test_dl)
    since = time.time() # timestamp for beginning of training

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    best_test_acc = 0.0
    best_f1 = 0.0
    best_precision = 0.0
    best_recall = 0.0
    
    phases = ['train', 'val', 'test'] if test_dl is not None else ['train', 'val']

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
                dl = dataloaders[train_dl]
                dataset_size = dataset_sizes[train_dl]                
            elif phase == 'val':
                model.eval()   # Set model to evaluate mode
                dl = dataloaders[val_dl]
                dataset_size = dataset_sizes[val_dl]
            else:
                model.eval()
                dl = dataloaders[test_dl]
                dataset_size = dataset_sizes[test_dl]

            running_loss = 0.0
            running_corrects = 0
            true_positives = 0.
            true_negatives = 0.
            false_positives = 0.
            false_negatives = 0.

            # Iterate over data.
            for inputs, labels in dl:
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.where(
                        outputs >= 0.,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                    target = torch.where(
                        labels >= 0.5,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device)
                    )
                    loss = criterion(outputs.view(target.shape), target)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                label_vals = torch.where(
                    labels >= 0.5,
                    torch.tensor([1.]).to(device),
                    torch.tensor([0.]).to(device)
                )
                correct = preds.view(label_vals.shape) == label_vals.data
                running_corrects += torch.sum(correct)
                
                true_positives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                true_negatives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_positives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_negatives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
            
            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size
            epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
            epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
            epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

            if verbose:
                print('{} Loss: {:.4f} Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc, epoch_pre, epoch_recall, epoch_f1))
                print('TP: {} TN: {} FP: {} FN: {}'.format(
                    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))
            if log_file is not None:
                log_file.write('Phase: {0}\t'
                               'Epoch: [{1}/{2}]\t'
                               'Loss: {loss_c:.4f}\t'
                               'Acc: {acc:.4f}\t'
                               'Pre: {pre:.4f}\t'
                               'Rec: {rec:.4f}\t'
                               'F1: {f1:.4f}\t'
                               'TP: {tp} '
                               'TN: {tn} '
                               'FP: {fp} '
                               'FN: {fn}\n'.format(
                                   phase, epoch + 1, num_epochs, loss_c=epoch_loss,
                                   acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                                   f1=epoch_f1, tp=int(true_positives.data), tn=int(true_negatives.data),
                                   fp=int(false_positives.data), fn=int(false_negatives.data)
                               ))
                log_file.flush()

            # deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_acc = epoch_acc
                best_f1 = epoch_f1
                best_precision = epoch_pre
                best_recall = epoch_recall
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test' and best_epoch == epoch:
                best_test_acc = epoch_acc

        time_elapsed = time.time() - since
        print('Training time {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print()

    # Print total training time
    time_elapsed = time.time() - since
    print('\nTraining complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    if return_best:
        print('Best epoch: {}'.format(best_epoch))
        print('Best val Acc: {:4f}'.format(best_acc))
        print('Best val Pre: {:4f}'.format(best_precision))
        print('Best val Rec: {:4f}'.format(best_recall))
        print('Best val F1: {:4f}'.format(best_f1))
        print('Test Acc: {:4f}'.format(best_test_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
    return model

In [14]:
path = os.path.join(data_root, 'Commercials\\models\\transfer_learning')
for seed in range(num_seeds):
    torch.manual_seed(seed)
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)

    model_ts = model_ts.to(device)

#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.]).to(device))
#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5.]).to(device))
    
    criterion = nn.BCEWithLogitsLoss()
    
    # Observe that all parameters are being optimized
    optimizer_ts = optim.SGD(model_ts.parameters(), lr=0.001, momentum=0.9)
    
    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler_ts = lr_scheduler.StepLR(optimizer_ts, step_size=7, gamma=0.1)
    
    if not os.path.exists(path):
        os.makedirs(path)
    with open(os.path.join(path, 'seed_{}.log'.format(seed)), 'w') as log_file:
        model_ts = train_model(model_ts, criterion, optimizer_ts, exp_lr_scheduler_ts,
                               'train', 'val', test_dl='test', num_epochs=global_num_epochs, verbose=True,
                               log_file=log_file, return_best=False)
        torch.save(model_ts.state_dict(), os.path.join(path, 'seed_{}.pth'.format(seed)))

train val test
Epoch 0/24
----------


C:\Users\mcram\Anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.3215 Acc: 0.8646 Pre: 0.7985 Rec: 0.7571 F1: 0.7772
TP: 3060.0 TN: 8141.0 FP: 772.0 FN: 982.0
val Loss: 0.2341 Acc: 0.9055 Pre: 0.7730 Rec: 0.9558 F1: 0.8547
TP: 2636.0 TN: 5947.0 FP: 774.0 FN: 122.0
test Loss: 0.1985 Acc: 0.9234 Pre: 0.8369 Rec: 0.9414 F1: 0.8861
TP: 2232.0 TN: 4690.0 FP: 435.0 FN: 139.0
Training time 12m 37s

Epoch 1/24
----------
train Loss: 0.2505 Acc: 0.9034 Pre: 0.8495 Rec: 0.8392 F1: 0.8443
TP: 3392.0 TN: 8312.0 FP: 601.0 FN: 650.0
val Loss: 0.1620 Acc: 0.9448 Pre: 0.8782 Rec: 0.9409 F1: 0.9085
TP: 2595.0 TN: 6361.0 FP: 360.0 FN: 163.0
test Loss: 0.1623 Acc: 0.9445 Pre: 0.9058 Rec: 0.9203 F1: 0.9130
TP: 2182.0 TN: 4898.0 FP: 227.0 FN: 189.0
Training time 24m 20s

Epoch 2/24
----------
train Loss: 0.2325 Acc: 0.9114 Pre: 0.8627 Rec: 0.8516 F1: 0.8571
TP: 3442.0 TN: 8365.0 FP: 548.0 FN: 600.0
val Loss: 0.1565 Acc: 0.9452 Pre: 0.8733 Rec: 0.9496 F1: 0.9098
TP: 2619.0 TN: 6341.0 FP: 380.0 FN: 139.0
test Loss: 0.1490 Acc: 0.9474 Pre: 0.9030 Rec: 0.9342 

val Loss: 0.2229 Acc: 0.9216 Pre: 0.8056 Rec: 0.9630 F1: 0.8773
TP: 2656.0 TN: 6080.0 FP: 641.0 FN: 102.0
test Loss: 0.1777 Acc: 0.9352 Pre: 0.8689 Rec: 0.9363 F1: 0.9013
TP: 2220.0 TN: 4790.0 FP: 335.0 FN: 151.0
Training time 273m 25s

Epoch 23/24
----------
train Loss: 0.1502 Acc: 0.9443 Pre: 0.9146 Rec: 0.9060 F1: 0.9103
TP: 3662.0 TN: 8571.0 FP: 342.0 FN: 380.0
val Loss: 0.1868 Acc: 0.9334 Pre: 0.8390 Rec: 0.9543 F1: 0.8930
TP: 2632.0 TN: 6216.0 FP: 505.0 FN: 126.0
test Loss: 0.1624 Acc: 0.9438 Pre: 0.9022 Rec: 0.9224 F1: 0.9122
TP: 2187.0 TN: 4888.0 FP: 237.0 FN: 184.0
Training time 285m 37s

Epoch 24/24
----------
train Loss: 0.1496 Acc: 0.9444 Pre: 0.9157 Rec: 0.9052 F1: 0.9104
TP: 3659.0 TN: 8576.0 FP: 337.0 FN: 383.0
val Loss: 0.1947 Acc: 0.9324 Pre: 0.8357 Rec: 0.9554 F1: 0.8916
TP: 2635.0 TN: 6203.0 FP: 518.0 FN: 123.0
test Loss: 0.1644 Acc: 0.9425 Pre: 0.8950 Rec: 0.9270 F1: 0.9107
TP: 2198.0 TN: 4867.0 FP: 258.0 FN: 173.0
Training time 297m 25s


Training complete in 297m 

# Smooth Results

In [15]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [16]:
path = os.path.join(data_root, 'Commercials\\models\\transfer_learning')
log_file_img = open(os.path.join(path, 'test_results_image_classifier.log'), 'w')
log_file_smoothed = open(os.path.join(path, 'test_results_smoothed.log'), 'w')

for seed in range(num_seeds):
    print('Working on seed:', seed)
    
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)
    model_ts.load_state_dict(torch.load(os.path.join(path, 'seed_{}.pth'.format(seed))))
    model_ts.to(device)
    criterion = nn.BCEWithLogitsLoss()
    
    model = model_ts.eval()   # Set model to evaluate mode
    dl = dataloaders['test']
    dataset_size = dataset_sizes['test']
    
    running_corrects = 0
    true_positives = 0.
    true_negatives = 0.
    false_positives = 0.
    false_negatives = 0.

    i = 0

    predictions = []
    gt_labels = []
    # Iterate over data.
    for inputs, labels in dl:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            preds = torch.where(
                outputs >= 0.,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
            target = torch.where(
                labels >= 0.5,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device)
            )

        predictions += preds.cpu().numpy().tolist()
        gt_labels += labels.cpu().numpy().tolist()

        # statistics
        label_vals = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        correct = preds.view(label_vals.shape) == label_vals.data
        running_corrects += torch.sum(correct)

        true_positives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        true_negatives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_positives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_negatives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )

        num_fp = torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        )

    #     if num_fp > 0:
    #         print(num_fp)
    #         print(torch.where(
    #             (correct == 0.) * (label_vals == 0.),
    #             torch.tensor([1.]).to(device),
    #             torch.tensor([0.]).to(device)))
    #         print(i)
    #         out = torchvision.utils.make_grid(inputs)
    # #         imshow(out, title=preds.cpu().numpy().tolist())
    #         imshow(out)

        i += 1

    epoch_acc = running_corrects.double() / dataset_size
    epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
    epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
    epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

    print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
        epoch_acc, epoch_pre, epoch_recall, epoch_f1))
    print('TP: {} TN: {} FP: {} FN: {}'.format(
        true_positives.data, true_negatives.data, false_positives.data, 
        false_negatives.data))
    
    log_file_img.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                       f1=epoch_f1, tp=int(true_positives.data),
                       tn=int(true_negatives.data),
                       fp=int(false_positives.data), fn=int(false_negatives.data)
                   ))
    log_file_img.flush()

    predictions = [p[0] for p in predictions]

    smoothed_preds = smooth_predictions(predictions, 3)

    print("Smoothed stats:")
    print(acc_prf1(smoothed_preds, gt_labels))
    
    sm_acc, sm_pre, sm_rec, sm_f1, sm_tp, sm_tn, sm_fp, sm_fn = acc_prf1(
        smoothed_preds, gt_labels)
    
    log_file_smoothed.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=sm_acc, pre=sm_pre, rec=sm_rec,
                       f1=sm_f1, tp=sm_tp,
                       tn=sm_tn,
                       fp=sm_fp, fn=sm_fn
                   ))
    log_file_smoothed.flush()

Working on seed: 0
Acc: 0.9425 Pre: 0.8950 Rec: 0.9270 F1: 0.9107
TP: 2198.0 TN: 4867.0 FP: 258.0 FN: 173.0
Smoothed stats:
(0.9401013874066169, 0.9004166666666666, 0.9114297764656263, 0.905889750576399, 2161, 4886, 239, 210)


# Understand Mistakes

In [18]:
path = os.path.join(data_root, 'Commercials\\models\\transfer_learning')
model_ts = models.resnet50(pretrained=True)
num_ftrs = model_ts.fc.in_features
model_ts.fc = nn.Linear(num_ftrs, 1)
model_ts.load_state_dict(torch.load(os.path.join(path, 'seed_0.pth')))
model_ts.to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3.]).to(device))

In [19]:
model = model_ts.eval()   # Set model to evaluate mode
dl = dataloaders['test']
dataset_size = dataset_sizes['test']

In [20]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.cpu().numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(str(title))
    plt.pause(0.001)  # pause a bit so that plots are updated

In [21]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [22]:
running_corrects = 0
true_positives = 0.
true_negatives = 0.
false_positives = 0.
false_negatives = 0.

i = 0

predictions = []
gt_labels = []
# Iterate over data.
for inputs, labels in dl:
    inputs = inputs.to(device)
    labels = labels.to(device).float()

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        preds = torch.where(
            outputs >= 0.,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        target = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        
    predictions += preds.cpu().numpy().tolist()
    gt_labels += labels.cpu().numpy().tolist()

    # statistics
    label_vals = torch.where(
        labels >= 0.5,
        torch.tensor([1.]).to(device),
        torch.tensor([0.]).to(device)
    )
    correct = preds.view(label_vals.shape) == label_vals.data
    running_corrects += torch.sum(correct)

    true_positives += torch.sum(
        torch.where(
            (correct == 1.) * (label_vals == 1.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    true_negatives += torch.sum(
        torch.where(
            (correct == 1.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    false_positives += torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    false_negatives += torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 1.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    
    num_fp = torch.sum(
    torch.where(
        (correct == 0.) * (label_vals == 0.),
        torch.tensor([1.]).to(device),
        torch.tensor([0.]).to(device))
    )

#     if num_fp > 0:
#         print(num_fp)
#         print(torch.where(
#             (correct == 0.) * (label_vals == 0.),
#             torch.tensor([1.]).to(device),
#             torch.tensor([0.]).to(device)))
#         print(i)
#         out = torchvision.utils.make_grid(inputs)
# #         imshow(out, title=preds.cpu().numpy().tolist())
#         imshow(out)
    
    i += 1

epoch_acc = running_corrects.double() / dataset_size
epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
    epoch_acc, epoch_pre, epoch_recall, epoch_f1))
print('TP: {} TN: {} FP: {} FN: {}'.format(
    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))

predictions = [p[0] for p in predictions]

smoothed_preds = smooth_predictions(predictions, 3)

print("Smoothed stats:")
print(acc_prf1(smoothed_preds, gt_labels))

Acc: 0.9448 Pre: 0.9086 Rec: 0.9178 F1: 0.9131
TP: 2176.0 TN: 4906.0 FP: 219.0 FN: 195.0
Smoothed stats:
(0.941435432230523, 0.90418410041841, 0.9114297764656263, 0.9077924805713086, 2161, 4896, 229, 210)
